## Full calculation of photoelectric heating in the WNM, CNM and CMM

    - Load the data for each grain in the integration and calculate the total heating rate.

In [25]:
import compute_charge_dist as fz
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

import PeHeat_Functions as peh
import parametric_fz as fzpar
import parametric_PeHeat as PeHpar
import dust_size_dist as sizedist

from matplotlib.colors import LogNorm

import pickle

import yt
from yt.units import pc, kpc, second, Kelvin, gram, erg, cm

import MCMCFit as mcmc

%matplotlib inline

microntocm    = 1.0e-4
cmtomicron    = 1.0e4
AAtocm        = 1.0e-8
cmtoAA        = 1.0e8
microntoAA    = 1.0e4
AAtomicron    = 1.0e-4
ergtoeV       = 6.242e11
eVtoerg       = 1.602e-12

hplanck       = 4.135667662e-15 # eV s
clight        = 2.99792458e10   # cm s-1

In [33]:
filename = "TotalHeating_ISM%s_%s_%.4iAA.pkl"%("CMM", "silicate", 7)
pkl_file = open("%s/%s"%("/home/jcibanezm/Dropbox/codes/run/PeHeat", filename), "rb")
cdist_sil= pickle.load(pkl_file)
pkl_file.close

IOError: [Errno 2] No such file or directory: '/home/jcibanezm/Dropbox/codes/run/PeHeat/TotalHeating_ISMCMM_silicate_0007AA.pkl'

In [35]:
pp = np.logspace(np.log10(3.5), np.log10(2500), num=100)

In [48]:
def get_Gamma_tot_Full_loadSavedData(phase=None, G0=1.7):
    """
    Returns GammaTot*1.0e21
    """
    
    import dust_size_dist as sizedist
    from scipy.integrate import simps
    
    data_dir = "/home/jcibanezm/Dropbox/codes/run/PeHeat"
    
    amin, amax = 3.5, 2500
    numint = 100
    x = np.logspace(np.log10(amin), np.log10(amax), num=numint)
    yc = sizedist.dnda(x, "carbonaceous")
    ys = sizedist.dnda(x, "silicate")

    if phase == None:
        print("")
        print("Need to define a phase ISM: WNM, CNM or CMM")
        print("")
    
    y_Gpe_s = np.zeros_like(x)
    y_Gpe_c = np.zeros_like(x)
    
    for i in range(len(x)):
        filename = "TotalHeating_ISM%s_%s_%.4iAA.pkl"%(phase, "silicate", x[i])
        pkl_file = open("%s/%s"%(data_dir, filename), "rb")
        cdist_sil= pickle.load(pkl_file)
        pkl_file.close

        y_Gpe_s[i] = cdist_sil["netHeating"][0]*ys[i]*1.0e21
        #print(y_Gpe_s[i])

        filename = "TotalHeating_ISM%s_%s_%.4iAA.pkl"%(phase, "carbonaceous", x[i])
        pkl_file = open("%s/%s"%(data_dir, filename), "rb")
        cdist_carb= pickle.load(pkl_file)
        pkl_file.close

        #print(cdist_carb)
        y_Gpe_c[i] = cdist_carb["netHeating"][0]*yc[i]*1.0e21

    Gamma_pe_sil_fix  = simps(y_Gpe_s, x)
    Gamma_pe_carb_fix = simps(y_Gpe_c, x)
    
    Gamma_tot= Gamma_pe_sil_fix + Gamma_pe_carb_fix

    print("G = ", cdist_sil["Gtot"], "T=", cdist_sil["temp"], "ne=", cdist_sil["ne"])
    
    return Gamma_tot/1.0e21

In [51]:
HeatingCMM = get_Gamma_tot_Full_loadSavedData("WNM")
#HeatingCNM = get_Gamma_tot_Full_loadSavedData("CNM")

IOError: [Errno 2] No such file or directory: '/home/jcibanezm/Dropbox/codes/run/PeHeat/TotalHeating_ISMWNM_silicate_0007AA.pkl'

In [3]:
HeatingCMM = get_Gamma_tot_Full_loadSavedData("CMM")
HeatingCNM = get_Gamma_tot_Full_loadSavedData("CNM")

('G = ', array([ 1.70079372]), 'T=', array([ 14.43397851]), 'ne=', array([[ 0.01077157],
       [ 0.00630768],
       [ 0.0105461 ]]))
('G = ', array([ 1.70561815]), 'T=', array([ 69.64352587]), 'ne=', array([[ 0.01077157],
       [ 0.00630768],
       [ 0.0105461 ]]))


In [14]:
print(HeatingCMM*3.0e4)
print(HeatingCNM*36.)

1.06221359521e-21
1.02355863112e-24


In [23]:
amin, amax, numint = 3.5, 1287.27137889, 90
x = np.logspace(np.log10(amin), np.log10(amax), num=numint)

In [24]:
print(x)

[    3.5            3.74020179     3.99688841     4.2711912      4.56431914
     4.87756417     5.21230693     5.57002278     5.95228833     6.36078843
     6.7973235      7.26381758     7.76232673     8.29504809     8.86432964
     9.47268046    10.12278183    10.81749907    11.55989411    12.35323905
    13.20103052    14.10700514    15.07515597    16.10975011    17.21534749
    18.396821      19.65937796    21.00858305    22.45038285    23.99113204
    25.63762144    27.39710789    29.27734629    31.28662373    33.43379603
    35.7283268     38.18032912    40.8006101     43.60071857    46.59299592
    49.79063051    53.20771584    56.8593126     60.7615151     64.93152217
    69.38771301    74.14972815    79.23855601    84.67662547    90.48790465
    96.69800659   103.33430213   110.42604056   118.00447849   126.10301767
   134.75735216   144.0056257    153.88859978   164.4498333    175.7358746
   187.79646657   200.68476591   214.45757741   229.17560435   244.90371598
   261.711233

In [16]:
print(x)

[    3.5            3.74020179     3.99688841     4.2711912      4.56431914
     4.87756417     5.21230693     5.57002278     5.95228833     6.36078843
     6.7973235      7.26381758     7.76232673     8.29504809     8.86432964
     9.47268046    10.12278183    10.81749907    11.55989411    12.35323905
    13.20103052    14.10700514    15.07515597    16.10975011    17.21534749
    18.396821      19.65937796    21.00858305    22.45038285    23.99113204
    25.63762144    27.39710789    29.27734629    31.28662373    33.43379603
    35.7283268     38.18032912    40.8006101     43.60071857    46.59299592
    49.79063051    53.20771584    56.8593126     60.7615151     64.93152217
    69.38771301    74.14972815    79.23855601    84.67662547    90.48790465
    96.69800659   103.33430213   110.42604056   118.00447849   126.10301767
   134.75735216   144.0056257    153.88859978   164.4498333    175.7358746
   187.79646657   200.68476591   214.45757742   229.17560435   244.90371598
   261.711233

In [5]:
# Parametric calculation 

In [6]:
phase = "CMM"
size = 1588
data_dir = "/home/jcibanezm/codes/run/PeHeat"
filename = "TotalHeating_ISM%s_%s_%.4iAA.pkl"%(phase, "carbonaceous", size)
pkl_file = open("%s/%s"%(data_dir, filename), "rb")
cdist    = pickle.load(pkl_file)
pkl_file.close

<function close>

In [7]:
cdist

{'Cooling': array([  1.85946892e-20]),
 'Geff': array([  1.15281812e-07]),
 'Gtot': array([ 0.00079384]),
 'NH2': array([  9.57334913e+21]),
 'Ntot': array([  1.23468728e+22]),
 'ZZ': array([-6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
        11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]),
 'ffz': array([  1.92944865e-33,   4.95455079e-22,   3.65948940e-13,
          9.01618687e-07,   9.13115949e-03,   5.27268882e-01,
          3.87815790e-01,   6.81516883e-02,   7.05996665e-03,
          5.37324667e-04,   3.25661530e-05,   1.64628313e-06,
          7.15543131e-08,   2.73208867e-09,   9.31211188e-11,
          2.86879944e-12,   8.06804574e-14,   2.08825410e-15,
          5.00826531e-17,   1.11936208e-18,   2.34298104e-20,
          4.61247430e-22,   8.57211321e-24,   1.50890762e-25,
          2.52308663e-27,   4.01825212e-29,   6.10949314e-31,
          8.88723125e-33,   1.23927432e-34,   1.65951584e-36,
          2.13755689e-38,   2.65234066e-40])

In [10]:
ParHeat_WNM = PeHpar.get_Gamma_tot_par(1.52, 7000., 0.9*0.012)*0.9
ParHeat_CNM = PeHpar.get_Gamma_tot_par(0.60, 70., 36.*1.8e-4)*36.
ParHeat_CMM = PeHpar.get_Gamma_tot_par(7.9e-4, 14.4, 3.0e4*9.0e-8)*3.0e4

In [11]:
print(ParHeat_WNM)
print(ParHeat_CNM)
print(ParHeat_CMM)

1.0626625824e-26
3.67994777476e-25
5.5997638738e-25
